In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import matplotlib.pyplot as plt
import numpy as np
import os

### Load the experiment and corresponding libraries

In [2]:
sys.path.append("/")  # enable import of "recipe"
os.chdir("/work/")

In [3]:
from recipes.main import load_experiment
experiment = load_experiment(recipe="_2020_09_10_benchmark", 
                             experiment="regression", 
                             schema="anix_nnfabrik_test",  # schema name will automatically be generated from recipe and experiment names 
                             base_dir="/recipes/",
                             import_prefix="recipes.", 
                             dev_mode=True  
                            )

CalledProcessError: Command '['/usr/local/bin/python3', '-m', 'pip', 'install', '-e', '/src/bias_transfer']' returned non-zero exit status 2.

In [4]:
from bias_transfer.tables.trained_model import *
from bias_transfer.tables.trained_transfer_model import *
schema

Connecting anix@134.2.168.16:3306


Schema `anix_nnfabrik_test`

### Run experiments

In [5]:
from recipes.main import fill_tables, run_all_experiments

In [6]:
fill_tables(experiment.transfer_experiments)  

In [7]:
run_all_experiments(experiment.transfer_experiments)

  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset: SVHN
Using downloaded and verified file: ./data/image_classification/torchvision/train_32x32.mat
Using downloaded and verified file: ./data/image_classification/torchvision/train_32x32.mat
Using downloaded and verified file: ./data/image_classification/torchvision/test_32x32.mat
Model with 11174475 parameters.
==> Starting model Noise Augmented + Repr. Matching (Euclid; clean only)!


Validation Epoch 0:   0%|          | 0/58 [00:00<?, ?it/s]/src/ml-utils/mlutils/tracking.py:144: RuntimeWarning: invalid value encountered in true_divide
  n_log[key] = l / norm
Validation Epoch 0: 100%|██████████| 58/58 [00:02<00:00, 24.75it/s, NoiseAdvTraining loss =0.0248, img_classification accuracy =75.4539, img_classification loss =0.8080]

Validation [001|-1/1000] ---> 75.45392491467577



Training Epoch 2:   0%|          | 0/516 [00:00<?, ?it/s]/src/ml-utils/mlutils/tracking.py:144: RuntimeWarning: invalid value encountered in true_divide
  n_log[key] = l / norm
Validation Epoch 0: 100%|██████████| 58/58 [00:02<00:00, 24.63it/s, NoiseAdvTraining loss =0.0248, img_classification accuracy =72.6416, img_classification loss =0.8825]

...

Validation [003|00/1000] ---> 80.83276450511946


Training Epoch 4:   0%|          | 0/516 [00:00<?, ?it/s]/src/ml-utils/mlutils/tracking.py:144: RuntimeWarning: invalid value encountered in true_divide
  n_log[key] = l / norm
Validation Epoch 0: 100%|██████████| 58/58 [00:02<00:00, 22.54it/s, NoiseAdvTraining loss =0.0248, img_classification accuracy =83.6177, img_classification loss =0.5470]

Validation [004|-1/1000] ---> 83.61774744027304



Training Epoch 5:   0%|          | 0/516 [00:00<?, ?it/s]/src/ml-utils/mlutils/tracking.py:144: RuntimeWarning: invalid value encountered in true_divide
  n_log[key] = l / norm
  0%|          | 0/1 [04:32<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
%debug

In [10]:
from nnfabrik.templates.trained_model_chkpts import Checkpoint

In [11]:
Checkpoint()

collapsed_key transfer,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,epoch epoch of creation,score current score at epoch,state current state,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion


In [10]:
# dElete all jobs in error state:
if schema.database != "nnfabrik_core": 
    (schema.jobs & "status='error'").delete()

In [48]:
# drop the entire schema: THIS IS WILL DELETE EVERYTHING!
if schema.database != "nnfabrik_core": 
    schema.drop()

Proceed to delete entire schema `anix_nnfabrik_test`? [yes, No]:  yes


In [46]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torch

experiment = "noisy_test"
run = "next run"
state = torch.load(Checkpoint().fetch("state",as_dict=True)[0]["state"])
tracker = state["tracker"]


In [47]:
def write_to_tensor_board(experiment, run, tracker):
    writer = SummaryWriter()
    def write_objective(obj_names, obj):
        if isinstance(obj,dict):
            for obj_name, sub_obj in obj.items():
                if obj_name == "normalization":
                    continue
                write_objective(obj_names+[obj_name],sub_obj)
        else:
            obj_val = tracker.get_objective(*obj_names)
            for i, val in enumerate(obj_val):
                writer.add_scalar('/'.join([run]+obj_names), val, i)
            
    write_objective([],tracker.objectives)
write_to_tensor_board(experiment, run, tracker)

/src/ml-utils/mlutils/tracking.py:144: RuntimeWarning: invalid value encountered in true_divide
  n_log[key] = l / norm


In [33]:
tracker.objectives

{'LR': 0,
 'Training': {'img_classification': {'loss': 0,
   'accuracy': 0,
   'normalization': 0},
  'NoiseAdvTraining': {'loss': 0, 'normalization': 0}},
 'Validation': {'img_classification': {'loss': 0,
   'accuracy': 0,
   'normalization': 0},
  'NoiseAdvTraining': {'loss': 0, 'normalization': 0}},
 'Test': {'NoiseAdvTraining': {'loss': 0, 'normalization': 0}}}

In [42]:
tracker.log

{'LR': [0.0003, 0.0003, 0.0003, 0],
 'Training': {'img_classification': {'loss': [312.89678955078125,
    477.7629699707031,
    316.83843994140625,
    0],
   'accuracy': [1900, 1900, 1100, 0],
   'normalization': [128, 128, 128, 0]},
  'NoiseAdvTraining': {'loss': [3.6585111916065216,
    5.0233617424964905,
    1.6073499619960785,
    0],
   'normalization': [60, 60, 60, 0]}},
 'Validation': {'img_classification': {'loss': [297.705322265625,
    298.799072265625,
    294.47955322265625,
    292.99310302734375],
   'accuracy': [2500, 1200, 3400, 2800],
   'normalization': [128, 128, 128, 128]},
  'NoiseAdvTraining': {'loss': [1.491750106215477,
    1.491749994456768,
    1.491749882698059,
    1.491750106215477],
   'normalization': [60, 60, 60, 60]}},
 'Test': {'NoiseAdvTraining': {'loss': [0, 0, 0, 0],
   'normalization': [0, 0, 0, 0]}}}

In [36]:
tracker.get_objective("LR")
tracker.get_objective("Training","img_classification","accuracy")

array([14.84375, 14.84375,  8.59375,      nan])

In [5]:
import torch
import numpy as np

In [6]:
from torchvision.datasets import SVHN
from torchvision import transforms

In [7]:
transform = transforms.Compose([
        transforms.ToTensor(),
#         transforms.Normalize((0.1307,), (0.3081,))
        ])
train = SVHN(
            root="/work/",
            split="train",
            download=True,
            transform=transform,
        )

Using downloaded and verified file: /work/train_32x32.mat


In [8]:
train_set = torch.utils.data.DataLoader(
        train,
        batch_size=64,
        shuffle=False,
    )
next(iter(train_set))[0].shape

torch.Size([64, 3, 32, 32])

In [65]:
mean = 0.
std = 0.
nb_samples = 0.
data_ = []
for data, _ in train_set:
    data_.append(data)
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples
dataset = torch.cat(data_)

In [66]:
print(mean,std)

tensor([0.4377, 0.4438, 0.4728]) tensor([0.1201, 0.1231, 0.1052])


In [67]:
std_ = dataset.std([0, 2, 3])
mean_ = dataset.mean([0, 2, 3])

In [68]:
std_

tensor([0.1980, 0.2010, 0.1970])

In [69]:
mean_

tensor([0.4377, 0.4438, 0.4728])

In [70]:
dataset.shape

torch.Size([73257, 3, 32, 32])